In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from transformers import RobertaModel


In [23]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "irony")

Found cached dataset tweet_eval (C:/Users/User/.cache/huggingface/datasets/tweet_eval/irony/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


  0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 2862
})

In [25]:
dataset["validation"]

Dataset({
    features: ['text', 'label'],
    num_rows: 955
})

In [26]:
dataset["validation"]

Dataset({
    features: ['text', 'label'],
    num_rows: 955
})

In [27]:
# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")


In [28]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

In [29]:
# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Get the input_ids from the tokenized dataset for train, validation, and test sets
train_input_ids = [example['input_ids'] for example in tokenized_dataset['train']]
val_input_ids = [example['input_ids'] for example in tokenized_dataset['validation']]
test_input_ids = [example['input_ids'] for example in tokenized_dataset['test']]

# Convert input_ids to PyTorch tensors
train_input_ids = torch.tensor(train_input_ids, dtype=torch.long)
val_input_ids = torch.tensor(val_input_ids, dtype=torch.long)
test_input_ids = torch.tensor(test_input_ids, dtype=torch.long)

# Concatenate the input_ids tensors along the first dimension
input_ids = torch.cat((train_input_ids, val_input_ids, test_input_ids), dim=0)


Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\irony\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-e2b5b11d4ece9e3b.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\irony\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-2fc35de4091337a6.arrow


Map:   0%|          | 0/955 [00:00<?, ? examples/s]

In [30]:
# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Get the attention_masks from the tokenized dataset for train, validation, and test sets
train_attention_mask = [example['attention_mask'] for example in tokenized_dataset['train']]
val_attention_mask = [example['attention_mask'] for example in tokenized_dataset['validation']]
test_attention_mask = [example['attention_mask'] for example in tokenized_dataset['test']]

# Convert attention_mask to PyTorch tensors
train_attention_mask = torch.tensor(train_attention_mask, dtype=torch.long)
val_attention_mask = torch.tensor(val_attention_mask, dtype=torch.long)
test_attention_mask = torch.tensor(test_attention_mask, dtype=torch.long)

# Concatenate the input_ids tensors along the first dimension
attention_masks = torch.cat((train_attention_mask, val_attention_mask, test_attention_mask), dim=0)


Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\irony\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-529c3886b9ca834c.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\irony\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-16d4cd1a1d1b920d.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\irony\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-0a6de7fafb93a330.arrow


In [31]:
print(input_ids)

tensor([[    0, 33471,   181,  ...,     1,     1,     1],
        [    0, 13724,    13,  ...,     1,     1,     1],
        [    0,  5975,    38,  ...,     1,     1,     1],
        ...,
        [    0,  1039, 12105,  ...,     1,     1,     1],
        [    0, 10431, 38398,  ...,     1,     1,     1],
        [    0, 10431, 37049,  ...,     1,     1,     1]])


In [32]:
print(attention_masks)

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [33]:
# Split the dataset into train, validation, and test sets
train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]
test_dataset = tokenized_dataset["test"]

In [34]:
# Convert the datasets to PyTorch tensors
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

In [35]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [36]:
class EmotionDetectionModel(nn.Module):
    def __init__(self, num_classes=2):
        super(EmotionDetectionModel, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.roberta.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits


In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [38]:
# instantiate your model
irony_model = EmotionDetectionModel(num_classes=2).to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
# define your loss function
criterion = nn.CrossEntropyLoss()

# define your optimizer
optimizer = torch.optim.Adam(irony_model.parameters(), lr=0.05)

In [40]:
# set up your training, validation, and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [41]:
for batch in train_loader:
    print(batch)


{'label': tensor([1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0]), 'input_ids': tensor([[    0, 30872,  6908,  ...,     1,     1,     1],
        [    0, 42577,   519,  ...,     1,     1,     1],
        [    0,  2387,  3267,  ...,     1,     1,     1],
        ...,
        [    0, 47519,     4,  ...,     1,     1,     1],
        [    0,    35,   623,  ...,     1,     1,     1],
        [    0, 10431, 15609,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'label': tensor([1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]), 'input_ids': tensor([[    0,   100,   657,  ...,     1,     1,     1],
        [    0, 15952,   341,  ...,     1,     1,     1],
        [    0,   243,    18,  ...,     1,     1,     1],
        ...,
        [    0,  1039, 12105,  ...,     1,     1,  

In [42]:
from tqdm import tqdm
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = irony_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # print running loss for each batch
        running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)
        avg_acc = correct_predictions / total_predictions
        tqdm.write(f'Train Loss: {avg_loss:.3f}, Train Acc: {avg_acc:.3f}', end='\r')
    tqdm.write(f'Epoch {epoch+1}, Train Loss: {avg_loss:.3f}, Train Acc: {avg_acc:.3f}')

    print(f"Epoch {epoch+1} finished")


Epoch 1/1:   1%|          | 1/179 [00:25<1:15:48, 25.55s/it]

Epoch 1/1:   1%|          | 2/179 [00:52<1:17:23, 26.24s/it]

Epoch 1/1:   2%|▏         | 3/179 [01:16<1:13:48, 25.16s/it]

Epoch 1/1:   2%|▏         | 4/179 [01:40<1:12:03, 24.70s/it]

Epoch 1/1:   3%|▎         | 5/179 [02:04<1:11:18, 24.59s/it]

...

Epoch 1/1:  98%|█████████▊| 176/179 [1:10:49<01:15, 25.04s/it]

Epoch 1/1:  99%|█████████▉| 177/179 [1:11:13<00:49, 24.93s/it]

Epoch 1/1:  99%|█████████▉| 178/179 [1:11:38<00:24, 24.85s/it]

Epoch 1/1: 100%|██████████| 179/179 [1:12:00<00:00, 24.14s/it]

Epoch 1, Train Loss: 4.235, Train Acc: 0.492
Epoch 1 finished


In [43]:
# Validation loop
with torch.no_grad():
    irony_model.eval()  # Set the model to evaluation mode
    valid_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in val_loader:
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # forward
        outputs = irony_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # calculate running loss
        valid_loss += loss.item()

    avg_loss = valid_loss / len(val_loader)
    avg_acc = correct_predictions / total_predictions
    print(f'Validation Loss: {avg_loss:.3f}, Validation Acc: {avg_acc:.3f}')

Validation Loss: 6.550, Validation Acc: 0.523


In [44]:

# Test loop
with torch.no_grad():
    irony_model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in test_loader:
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # forward
        outputs = irony_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # calculate running loss
        test_loss += loss.item()

    avg_loss = test_loss / len(test_loader)
    avg_acc = correct_predictions / total_predictions
    print(f'Test Loss: {avg_loss:.3f}, Test Acc: {avg_acc:.3f}')


Test Loss: 5.449, Test Acc: 0.603


In [45]:
# save the model
torch.save(irony_model.state_dict(), 'Models/irony_model.pth')
